## Imports

In [17]:
import matplotlib.pyplot as plt
from scipy.odr import *
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/artur/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['exponential']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


## Opening directory

In [ ]:
%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/

## Fitting power law to Rtr vs. Mdot

In [35]:
rtr = [11.2, 13.2, 20.3, 27.5]
rtr_err = [2.5, 2.7, 5.5, 5.8]
mdot = [0.35, 0.24, 0.11, 0.06]
mdot_err = [0.09, 0.07, 0.02, 0.01]
luminosity = [0.82, 0.51, 0.22, 0.08]
luminosity_err = [0.08, 0.04, 0.01, 0.01]
Lanal = [11.4, 5.2, 1.3, 0.3]
Lssanal = [0.07, 0.04, 0.01, 0.004]

In [21]:
def Powerlaw(a, x):
    return (a[0] * x**a[1])

In [ ]:
PL_model = Model(Powerlaw)

data = RealData(luminosity, rtr, sx=luminosity_err, sy=rtr_err)

myodr = ODR(data, PL_model, beta0=[3., -0.5])

myoutput = myodr.run()

myoutput.pprint()

#myoutput.beta = best values of parameters
#myoutput.sd_beta = standard deviation of parameters

In [ ]:
plt.figure(figsize=(8,6))

plt.errorbar(x=luminosity, y=rtr, yerr=rtr_err, xerr=luminosity_err,
             fmt='bo',
             capsize=3)

xfit = np.linspace(0.05, 1.0, num=100)
yfit = Powerlaw(myoutput.beta, xfit)
#plusyfit = Powerlaw(myoutput.beta + myoutput.sd_beta, xfit)
#minusyfit = Powerlaw(myoutput.beta - myoutput.sd_beta, xfit)

plt.plot(xfit, yfit, '--r', 
         label = '$a = %0.2f \pm %0.2f$\n$b = %0.2f \pm %0.2f$' %
         (myoutput.beta[0], myoutput.sd_beta[0], myoutput.beta[1], myoutput.sd_beta[1]))

#plt.fill_between(xfit, minusyfit, plusyfit,
#                 color='r', alpha=0.1)

#plt.title("$Rtr = a(\dot{M}/\dot{M}_{\\rm Edd})^{b}$", fontsize=16)
plt.title("$Rtr = a(L/L_{\\rm Edd})^{b}$", fontsize=16)

plt.xlabel("$L/L_{\\rm Edd}$", fontsize=15)
#plt.xlabel("$\dot{M}/\dot{M}_{\\rm Edd}$", fontsize=15)
plt.ylabel("$R_{\\rm tr}/R_s$", fontsize=15)

plt.legend(fontsize=13)
#plt.xscale('log')
#plt.yscale('log')

plt.tick_params(axis='both', which='major', labelsize=13)
plt.tick_params(axis='both', which='minor', labelsize=12)
plt.tight_layout()

plt.savefig('rtr_vs_L.png', dpi=300)
plt.clf()
plt.close('all')

## Sanity check of luminosity

In [ ]:
import numpy as np
import mickey.mickey
import mickey.plot
import tqdm

In [ ]:
#%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho5e-6/
#%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho1.5e-6/
#%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho1e-6/
#%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho5e-7/
%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho3e-7/
#%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho5e-8/
#%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho1e-8/
#%cd /home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rho5e-9/

In [ ]:
#Mass of black hole in solar mass
mbh = 10.0

Ledd = 1.26e38 * mbh
Medd = 1.39916e18 * mbh

#units of sim
unit_length = 2.969707e6
unit_velocity = 2.1198528e10
#MOST IMPORTANT UNIT TO MY WORK
unit_density = 3e-7

c = unit_velocity * np.sqrt(2.)

#Conversion of mdot from code units to cgs
mdot_conv = unit_length*unit_length*unit_density*unit_velocity

In [2]:
def C(x):
    return (3./x - 2.* x**(-1.5))

In [64]:
def eta(x):
    return (1./(4.*x)*(1 - C(2000./x)))

In [ ]:
data = np.load('data.npz')
#print (data['arr_0'])
mdot_ss = data['arr_0']*Medd
Rtr = data['arr_1']

In [ ]:
# will hold mdot-values at the inner boundary
mdot_riaf=[]
mdot_ss=[]

isnap = 2123 #2634
fsnap = 2478 #2989

for i in tqdm.tqdm(range(isnap, fsnap)):
    # reads data
    p=mickey.mickey.Pluto(i, stdout=False)
    
    mdot_riaf.append(p.mdotr(3.))
    mdot_ss.append(p.mdotr(Rtr[i-isnap]))

In [ ]:
mdot_ss = np.array(mdot_ss)
mdot_ss *= mdot_conv

mdot_riaf = np.array(mdot_riaf)
mdot_riaf *= mdot_conv

In [ ]:
mdot_ss = abs(mdot_ss)
mdot_riaf = abs(mdot_riaf)

In [ ]:
# Riaf Luminosity
Lriaf = 10 * mdot_riaf * mdot_riaf * c * c / Medd

In [ ]:
# Thin disc luminosity
Lss = eta(Rtr) * mdot_ss * c * c

In [ ]:
L = (np.median(Lss))/Ledd

In [ ]:
print(L)

In [4]:
def mass_accretion_rate_Reb2018(luminosity, truncation_radius):
    mbh = 10.0
    Ledd = 1.26e38 * mbh
    Medd = 1.39916e18 * mbh
    light_speed_squared = 2.99792458e10 * 2.99792458e10
    
    return (luminosity/eta(2*truncation_radius)/light_speed_squared * Ledd/Medd)

In [65]:
garcia_luminosity = np.array([0.016, 0.039, 0.079, 0.119, 0.142, 0.173])
garcia_rtr = np.array([3.7, 2.7, 2.1, 1.8, 1.5, 1.7])*1.237
garcia_rtr_err = np.array([[1.0, 1.5, 0.5, 0.2, 0.2, 0.3],[0.7, 0.5, 0.3, 0.1, 0.4, 0.2]])*1.237

In [66]:
garcia_mdot = mass_accretion_rate_Reb2018(garcia_luminosity, garcia_rtr)

In [67]:
print(garcia_mdot)

[0.05948056 0.10542751 0.16574854 0.21377641 0.21235158 0.29341391]


In [68]:
PL_model = Model(Powerlaw)

data = RealData(garcia_mdot, garcia_rtr, sy=garcia_rtr_err[1])

myodr = ODR(data, PL_model, beta0=[3., -0.5])

myoutput = myodr.run()

myoutput.pprint()

#myoutput.beta = best values of parameters
#myoutput.sd_beta = standard deviation of parameters

Beta: [ 0.70613214 -0.728428  ]
Beta Std Error: [0.29344271 0.2609153 ]
Beta Covariance: [[42.04073648 36.81646255]
 [36.81646255 33.23707241]]
Residual Variance: 0.0020482187019414764
Inverse Condition #: 0.0856455815951856
Reason(s) for Halting:
  Sum of squares convergence


In [37]:
PL_model2 = Model(Powerlaw)

data2 = RealData(mdot, rtr, sx=mdot_err, sy=rtr_err)

myodr2 = ODR(data2, PL_model2, beta0=[3., -0.5])

myoutput2 = myodr2.run()

myoutput2.pprint()

#myoutput.beta = best values of parameters
#myoutput.sd_beta = standard deviation of parameters

Beta: [ 6.42568747 -0.51702729]
Beta Std Error: [0.15319595 0.01163271]
Beta Covariance: [[5.54190241 0.39458356]
 [0.39458356 0.03195409]]
Residual Variance: 0.004234827307511398
Inverse Condition #: 0.025228125338655587
Reason(s) for Halting:
  Sum of squares convergence


In [63]:
figure(figsize=(8,5))
plt.errorbar(garcia_mdot, garcia_rtr, garcia_rtr_err, np.zeros(len(garcia_mdot)), fmt='bo', capsize=3)
plt.errorbar(mdot, rtr, rtr_err, mdot_err, fmt='gs', capsize=3)

xfit = np.linspace(0.04, 1.0, num=100)
yfit = Powerlaw(myoutput.beta, xfit)
plt.plot(xfit, yfit, '--r', 
         label = r'$R_{\mathrm{tr}} \propto \dot{m}^{%0.2f \pm %0.2f}$' %
         (myoutput.beta[1], myoutput.sd_beta[1]))

xfit2 = np.linspace(0.04, 1.0, num=100)
yfit2 = Powerlaw(myoutput2.beta, xfit)
plt.plot(xfit2, yfit2, '-.k', 
         label = r'$R_{\mathrm{tr}} \propto \dot{m}^{%0.2f \pm %0.2f}$' %
         (myoutput2.beta[1], myoutput2.sd_beta[1]))
xlabel("$\dot{M}/\dot{M}_{\\rm Edd}$", fontsize=15)
ylabel("$R_{\mathrm{tr}}/R_{\mathrm{s}}$", fontsize=15)
xscale("log")
yscale("log")
ylim(1, 90.)
xlim(0.04, 0.5)
tick_params(axis='both', which='major', labelsize=13)
tick_params(axis='both', which='minor', labelsize=12)

plt.legend(fontsize=15)
tight_layout()

#plt.show()
plt.savefig('/home/artur/Desktop/Torus_pn_2D_cooling/Simulations/rtr-mdot-comparison.png', dpi=300)
plt.clf()
plt.close('all')